In [1]:
import numpy as np
import os
import pathlib
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential

In [4]:
# Define constants
IMAGE_SHAPE = (200, 200)  # Resize all images to this shape for MobileNetV2
BATCH_SIZE = 100
DATA_DIR = 'F:/indhu/Datasets/dog_breed_30_images/'  # Update with your dataset path


In [5]:
# Load data
data_dir = pathlib.Path(DATA_DIR)
breeds = os.listdir(DATA_DIR)
dogs_images_dict = {}
dogs_labels_dict = {}
X, y = [], []
for index, category in enumerate(breeds):
    dogs_images_dict[category] = list(data_dir.glob(category + '/*'))
    dogs_labels_dict[category] = index

for breed_name, images in dogs_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(dogs_labels_dict[breed_name])

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)


In [6]:

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Create MobileNetV2 feature extractor
base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')
base_model_mobilenet.trainable = False  # Freeze the convolutional base

feature_extractor_mobilenet = Sequential([
    base_model_mobilenet,
    GlobalAveragePooling2D()
])

C:\Users\Indhuma\AppData\Local\Temp\ipykernel_8572\485066812.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:

# Extract features using MobileNetV2
X_train_features = feature_extractor_mobilenet.predict(X_train, batch_size=BATCH_SIZE, verbose=1)
X_test_features = feature_extractor_mobilenet.predict(X_test, batch_size=BATCH_SIZE, verbose=1)


39/39 ━━━━━━━━━━━━━━━━━━━━ 28s 682ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 637ms/step


In [9]:
# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine": SVC(kernel='rbf', probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}

In [ ]:
# Evaluate classifiers using test set
for model_name, model in classifiers.items():
    print(f"Evaluating {model_name}...")
    model.fit(X_train_features, y_train)
    y_pred = model.predict(X_test_features)
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

Evaluating Random Forest...
Random Forest Accuracy: 0.7299
              precision    recall  f1-score   support

           0       0.66      0.61      0.63        41
           1       0.53      0.25      0.34        40
           2       0.75      0.83      0.79        29
           3       0.93      0.76      0.84        34
           4       0.67      0.42      0.52        33
           5       0.52      0.69      0.59        32
           6       0.90      0.90      0.90        39
           7       0.81      0.62      0.70        34
           8       0.51      0.58      0.55        31
           9       0.59      0.86      0.70        28
          10       0.96      0.93      0.95        28
          11       0.71      0.79      0.75        28
          12       0.70      0.64      0.67        25
          13       0.76      0.79      0.78        24
          14       0.76      0.90      0.82        31
          15       0.80      0.90      0.85        40
          16       0.8

In [12]:
# Evaluate classifiers using k-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for clf_name, clf in classifiers.items():
    print(f"K-Fold Cross Validation for {clf_name}...")
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for train_index, val_index in kf.split(X_train_features, y_train):
        X_train_fold, X_val_fold = X_train_features[train_index], X_train_features[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        clf.fit(X_train_fold, y_train_fold)
        y_val_pred = clf.predict(X_val_fold)

        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
        recall_scores.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
        f1_scores.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

    print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
    print(f"Average Precision: {np.mean(precision_scores):.4f}")
    print(f"Average Recall: {np.mean(recall_scores):.4f}")
    print(f"Average F1 Score: {np.mean(f1_scores):.4f}\n")


K-Fold Cross Validation for Random Forest...
Average Accuracy: 0.7341
Average Precision: 0.7401
Average Recall: 0.7341
Average F1 Score: 0.7315

K-Fold Cross Validation for Logistic Regression...
Average Accuracy: 0.8011
Average Precision: 0.8059
Average Recall: 0.8011
Average F1 Score: 0.8002

K-Fold Cross Validation for K-Nearest Neighbors...
Average Accuracy: 0.7469
Average Precision: 0.7601
Average Recall: 0.7469
Average F1 Score: 0.7450

K-Fold Cross Validation for Support Vector Machine...
Average Accuracy: 0.8037
Average Precision: 0.8176
Average Recall: 0.8037
Average F1 Score: 0.8046

K-Fold Cross Validation for Decision Tree...
Average Accuracy: 0.4226
Average Precision: 0.4270
Average Recall: 0.4226
Average F1 Score: 0.4207

K-Fold Cross Validation for Naive Bayes...
Average Accuracy: 0.7218
Average Precision: 0.7466
Average Recall: 0.7218
Average F1 Score: 0.7257

